## FIT3182 - Assignment 2
---
### Part A

**Information:**
- Filename: Assignment_PartA.ipynb
- Student Name: Nicholas Mandylas
- Student Number: 27840328
- Student Email: nman48@student.monash.edu

---

### Task 1

**Q1:** 

Based on the two data sets provided i.e. hotspot_historic.csv and climate_historic.csv, design a suitable data model to support the efficient quering of the two data sets in MongoDB. Justify your data model design. 
  - An example of the data model.
  - The justification for choosing that data model.

**A1:** 

Justification: 

The first step in the process is to look at the indvidual datasets, and identify any relationships that occur between each dataset. Looking at the dataset 'climate_historic', we can observe that only one record is produced per day. Whilst on the otherhand, we can observe that the dataset 'hotspot_historic' can have zero to many records on a single day. As a result, using the date as the connecting data point between to the two collections, we can model a one-to-many relationship, where a climate record, can have many hotspot records.

Taking into consideration above, I've choosen to create an embedded database model. Embedding the hotspot data into a document, will reduce the number of read operations required to obtain the data. This allows the model to be more efficient, as the general principle is to request all information with the least number of read operations as possible. Thus, this eliminates the needs for joins, which can be quite a taxing operation to perform. Another important benefit, is that operations peformed on a embedded model is always atomic, whilst in a reference model, atomic operations can never be assumed.

A potential problem with this approach is that this can lead to large documents, especially when in this case, a 'climate' may have a large subset of 'hotspots'. This won't be an issue  
MongoDB has a 4MB size limit on a single document, which more than enough to account for 2000+ hotspot records in the dataset, which will be split up into each individual climate. The 4MB limit is the equivalent of holding 30,000+ comments on a webpage, which in this case, there isn't a realistic possibility of 30,000+ widespread fires occurring accross Victoria on a single given day. 

Model:

```
{ 
  '_id': ObjectId('1a2b3c4d5f6g7h8i9j0k'),
  'date': '2/11/2018',
  'station': 948700,
  'air_temperature_celcius': 15,
  'relative_humidity': 50.7,
  'windspeed_knots': 9.2,
  'max_wind_speed': 13,
  'precipitation': 0.02,
  'precipitation_type': 'G',
  'ghi': 128,
  'hotspots': [
    {
      'time': '04:16:51',
      'latitude': -37.966,
      'longitude': 145.051,
      'confidence': 78,
      'surface_temperature_celcius': 68,
    },
    { 
      'time': '05:13:31',
      'latitude': -34.966,
      'longitude': 144.051,
      'confidence': 91,
      'surface_temperature_celcius': 72,
    },
  ]
}
```


---

### Task 2

**Q1:** 

Write a python program that will read the data from hotspot_historic.sv and climate_historic.csv and load them to the new database (e.g. fit3182_assignment_db). The collections will be based on the document model shown above.


In [33]:
from datetime import datetime
from pprint import pprint
import pymongo
import pandas

# Connect to MongoClient & get/create assignment database
client = pymongo.MongoClient()
db = client.fit3182_assignment_db
collection = db.climate

# Drop existing collection, to reload data from CSV
collection.drop()

# Read data from CSV using pandas
hotspot_data = pandas.read_csv('hotspot_historic.csv')
climate_data = pandas.read_csv('climate_historic.csv')

collection_data = []

for index, row in climate_data.iterrows():
    document = {}

    # Format data appropriately for document object.
    document['date'] = datetime.strptime(row['date'], '%d/%m/%y')
    document['station'] = int(row['station'])
    document["air_temperature_celcius"] = float(
        row["air_temperature_celcius"])
    document['relative_humidity'] = float(row['relative_humidity'])
    document['windspeed_knots'] = float(row['windspeed_knots'])
    document['max_wind_speed'] = float(row['max_wind_speed'])

    # Unncessary space at beginning of value is removed.
    # Also split precipation type and amount, to make it easier for sorting/searching later.
    precipitation = str(row['precipitation ']).replace(" ", "")
    document['precipitation_type'] = precipitation[-1]

    document['precipitation'] = float(precipitation[0:-1])
    document['ghi'] = float(row['GHI_w/m2'])

    # Get all hotspot records that match climate date & append to document.
    hotspot_date = document['date'].strftime("%-d/%m/%Y")
    hotspots = hotspot_data[hotspot_data['date'] == hotspot_date]

    # If there is a match, we clean up data & append records to document.
    if len(hotspots) > 0:
        document['hotspots'] = []
        for index, row in hotspots.iterrows():
            hotspot = {}
            hotspot['time'] = datetime.strptime(
                row['datetime'], '%Y-%m-%dT%H:%M:%S')
            hotspot['latitude'] = float(row['latitude'])
            hotspot['longitude'] = float(row['longitude'])
            hotspot['confidence'] = float(row['confidence'])
            hotspot['surface_temperature_celcius'] = float(
                row['surface_temperature_celcius'])
            document['hotspots'].append(hotspot)

    collection_data.append(document)

# Insert collection into database
collection.insert_many(collection_data)


---

### Task 2

**Q2:** 

Write queries to answer the following tasks on fit3182_assignment_db and corresponding collection(s). You need to write the queries as python program using the pymongo library in Jupyter Notebook.


A. Find climate data on 12th December 2018.

In [34]:
result = collection.find_one({'date': datetime(2018, 12, 12)})
pprint(result)

{'_id': ObjectId('60aa4d77f1c7e6844f090d16'),
 'air_temperature_celcius': 19.0,
 'date': datetime.datetime(2018, 12, 12, 0, 0),
 'ghi': 156.0,
 'hotspots': [{'confidence': 53.0,
               'latitude': -37.903,
               'longitude': 145.25,
               'surface_temperature_celcius': 44.0,
               'time': datetime.datetime(2018, 12, 12, 0, 45, 38)}],
 'max_wind_speed': 12.0,
 'precipitation': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


B. Find latitude, longitude, surface temperature, and confidence when the surfance temperature was between 65 and 100.

In [35]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {'$match': {'hotspots.surface_temperature_celcius': {'$gte': 65, '$lte': 100}}}, {
    '$project': {'_id': 0, 'hotspots.longitude': 1, 'hotspots.latitude': 1, 'hotspots.surface_temperature_celcius': 1}}])

for each in result:
    pprint(each)


{'hotspots': {'latitude': -37.2284,
              'longitude': 147.9187,
              'surface_temperature_celcius': 73.0}}
{'hotspots': {'latitude': -37.6572,
              'longitude': 142.0703,
              'surface_temperature_celcius': 80.0}}
{'hotspots': {'latitude': -37.0193,
              'longitude': 148.1459,
              'surface_temperature_celcius': 71.0}}
{'hotspots': {'latitude': -37.4229,
              'longitude': 147.027,
              'surface_temperature_celcius': 99.0}}
{'hotspots': {'latitude': -37.0055,
              'longitude': 148.1582,
              'surface_temperature_celcius': 68.0}}
{'hotspots': {'latitude': -37.4128,
              'longitude': 147.0242,
              'surface_temperature_celcius': 98.0}}
{'hotspots': {'latitude': -34.357,
              'longitude': 141.5361,
              'surface_temperature_celcius': 67.0}}
{'hotspots': {'latitude': -34.3539,
              'longitude': 141.5547,
              'surface_temperature_celcius': 72.0}}
{'

C. Find date, surface temperature, air temperature, relative humidity and max wind speed on 15th and 16th of December 2018.

In [36]:
result = collection.find({'date': {'$in': [datetime(2018, 12, 15), datetime(2018, 12, 16)]}}, {
    '_id': 0, 'date': 1, 'hotspots.surface_temperature_celcius': 1, 'air_temperature_celcius': 1, 'relative_humidity': 1, 'max_wind_speed': 1})

for each in result:
    pprint(each)

{'air_temperature_celcius': 18.0,
 'date': datetime.datetime(2018, 12, 15, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 42.0},
              {'surface_temperature_celcius': 36.0},
              {'surface_temperature_celcius': 38.0},
              {'surface_temperature_celcius': 40.0}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18.0,
 'date': datetime.datetime(2018, 12, 16, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 43.0},
              {'surface_temperature_celcius': 33.0},
              {'surface_temperature_celcius': 54.0},
              {'surface_temperature_celcius': 73.0},
              {'surface_temperature_celcius': 55.0},
              {'surface_temperature_celcius': 75.0},
              {'surface_temperature_celcius': 55.0},
              {'surface_temperature_celcius': 66.0},
              {'surface_temperature_celcius': 56.0},
              {'surface_temperature_celcius': 60.0},
              {'surface_temperature_celc

D. Find datetime, air temperature, surface temperature and confidence when the confidence is between 80 and 100.

In [37]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {'$match': {'hotspots.confidence': {'$gte': 80, '$lte': 100}}}, {
    '$project': {'_id': 0, 'hotspots.time': 1, 'air_temperature_celcius': 1, 'hotspots.surface_temperature_celcius': 1, 'hotspots.confidence': 1}}])

for each in result:
    pprint(each)

{'air_temperature_celcius': 20.0,
 'hotspots': {'confidence': 87.0,
              'surface_temperature_celcius': 62.0,
              'time': datetime.datetime(2018, 3, 6, 5, 6, 30)}}
{'air_temperature_celcius': 20.0,
 'hotspots': {'confidence': 85.0,
              'surface_temperature_celcius': 59.0,
              'time': datetime.datetime(2018, 3, 6, 5, 6, 20)}}
{'air_temperature_celcius': 19.0,
 'hotspots': {'confidence': 88.0,
              'surface_temperature_celcius': 64.0,
              'time': datetime.datetime(2018, 3, 7, 4, 16, 10)}}
{'air_temperature_celcius': 23.0,
 'hotspots': {'confidence': 86.0,
              'surface_temperature_celcius': 41.0,
              'time': datetime.datetime(2018, 3, 9, 13, 23, 40)}}
{'air_temperature_celcius': 19.0,
 'hotspots': {'confidence': 100.0,
              'surface_temperature_celcius': 105.0,
              'time': datetime.datetime(2018, 3, 10, 4, 48, 40)}}
{'air_temperature_celcius': 19.0,
 'hotspots': {'confidence': 100.0,
         

E. Find the top 10 records with the highest surface temperature.

In [26]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {
    '$sort': {'hotspots.surface_temperature_celcius': -1}}, {'$limit': 10}])

for each in result:
    pprint(each)

{'_id': ObjectId('60aa3feaf0d4af2cc0bd6ca6'),
 'air_temperature_celcius': 15.0,
 'date': datetime.datetime(2018, 4, 18, 0, 0),
 'ghi': 122.0,
 'hotspots': {'confidence': 100.0,
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124.0,
              'time': datetime.datetime(2018, 4, 18, 4, 52)},
 'max_wind_speed': 9.9,
 'precipitation': '0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'_id': ObjectId('60aa3feaf0d4af2cc0bd6c98'),
 'air_temperature_celcius': 16.0,
 'date': datetime.datetime(2018, 4, 4, 0, 0),
 'ghi': 140.0,
 'hotspots': {'confidence': 100.0,
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123.0,
              'time': datetime.datetime(2018, 4, 4, 4, 32, 50)},
 'max_wind_speed': 12.0,
 'precipitation': '0.00I',
 'relative_humidity': 47.5,
 'station': 948701,
 'windspeed_knots': 5.4}
{'_id': ObjectId('60aa3

F. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.

In [38]:
result = collection.aggregate(
    [{'$project': {'_id': 0, 'date': 1, 'numberOfFires': {'$size': {'$ifNull': ['$hotspots', []]}}}}])

for each in result:
    pprint({
        "date": each['date'].strftime("%d/%m/%Y"),
        "numberOfFires": each['numberOfFires']
    })

{'date': '31/12/2017', 'numberOfFires': 0}
{'date': '02/01/2018', 'numberOfFires': 0}
{'date': '03/01/2018', 'numberOfFires': 0}
{'date': '04/01/2018', 'numberOfFires': 0}
{'date': '05/01/2018', 'numberOfFires': 0}
{'date': '06/01/2018', 'numberOfFires': 0}
{'date': '07/01/2018', 'numberOfFires': 0}
{'date': '08/01/2018', 'numberOfFires': 0}
{'date': '09/01/2018', 'numberOfFires': 0}
{'date': '10/01/2018', 'numberOfFires': 0}
{'date': '11/01/2018', 'numberOfFires': 0}
{'date': '12/01/2018', 'numberOfFires': 0}
{'date': '13/01/2018', 'numberOfFires': 0}
{'date': '14/01/2018', 'numberOfFires': 0}
{'date': '15/01/2018', 'numberOfFires': 0}
{'date': '16/01/2018', 'numberOfFires': 0}
{'date': '17/01/2018', 'numberOfFires': 0}
{'date': '18/01/2018', 'numberOfFires': 0}
{'date': '19/01/2018', 'numberOfFires': 0}
{'date': '20/01/2018', 'numberOfFires': 0}
{'date': '21/01/2018', 'numberOfFires': 0}
{'date': '22/01/2018', 'numberOfFires': 0}
{'date': '23/01/2018', 'numberOfFires': 0}
{'date': '2

G. Find the records of fires where the confidence is below 70.

In [39]:
result = collection.aggregate(
    [{'$unwind': '$hotspots'}, {'$match': {'hotspots.confidence': {'$lt': 70}}}, {'$project': {'_id': 0, 'hotspots': 1}}])

for each in result:
    pprint(each)

{'hotspots': {'confidence': 68.0,
              'latitude': -37.7885,
              'longitude': 141.9352,
              'surface_temperature_celcius': 55.0,
              'time': datetime.datetime(2018, 3, 8, 4, 51)}}
{'hotspots': {'confidence': 54.0,
              'latitude': -37.7171,
              'longitude': 147.5866,
              'surface_temperature_celcius': 44.0,
              'time': datetime.datetime(2018, 3, 9, 3, 57)}}
{'hotspots': {'confidence': 55.0,
              'latitude': -36.2544,
              'longitude': 148.0353,
              'surface_temperature_celcius': 42.0,
              'time': datetime.datetime(2018, 3, 10, 4, 43)}}
{'hotspots': {'confidence': 54.0,
              'latitude': -37.2197,
              'longitude': 147.9621,
              'surface_temperature_celcius': 43.0,
              'time': datetime.datetime(2018, 3, 10, 4, 42, 30)}}
{'hotspots': {'confidence': 56.0,
              'latitude': -37.0286,
              'longitude': 148.155,
            

H. Find the average surface temperature for each day. You are required to only display average surface temperature and the date in the output.

In [31]:
result = collection.aggregate([{'$unwind': '$hotspots'}, {'$group': {
    '_id': '$date', 'avgTemp': {'$avg': '$hotspots.surface_temperature_celcius'}}}])

for each in result:
    pprint({
        'date': each['_id'].strftime("%d/%m/%Y"),
        'avgTemp': each['avgTemp']
    })

{'avgTemp': 49.0, 'date': '14/05/2018'}
{'avgTemp': 51.2, 'date': '23/05/2018'}
{'avgTemp': 69.375, 'date': '10/03/2018'}
{'avgTemp': 50.6, 'date': '08/12/2018'}
{'avgTemp': 56.291666666666664, 'date': '08/05/2018'}
{'avgTemp': 56.5, 'date': '10/09/2018'}
{'avgTemp': 46.0, 'date': '22/06/2018'}
{'avgTemp': 38.5, 'date': '13/03/2018'}
{'avgTemp': 60.925925925925924, 'date': '28/03/2018'}
{'avgTemp': 47.0, 'date': '31/07/2018'}
{'avgTemp': 47.0, 'date': '13/11/2018'}
{'avgTemp': 62.57303370786517, 'date': '04/04/2018'}
{'avgTemp': 52.0, 'date': '14/11/2018'}
{'avgTemp': 61.94444444444444, 'date': '14/04/2018'}
{'avgTemp': 45.0, 'date': '22/09/2018'}
{'avgTemp': 44.333333333333336, 'date': '30/06/2018'}
{'avgTemp': 52.69565217391305, 'date': '12/04/2018'}
{'avgTemp': 53.89473684210526, 'date': '23/04/2018'}
{'avgTemp': 53.333333333333336, 'date': '10/10/2018'}
{'avgTemp': 50.4, 'date': '27/10/2018'}
{'avgTemp': 58.5, 'date': '05/11/2018'}
{'avgTemp': 56.80740740740741, 'date': '04/05/2018

i. Find the top 10 records with the lowest GHI.

In [40]:
result = collection.aggregate([{'$sort': {'ghi': 1}}, {'$limit': 10}])

for each in result:
    pprint(each)

{'_id': ObjectId('60aa4d77f1c7e6844f090c92'),
 'air_temperature_celcius': 5.0,
 'date': datetime.datetime(2018, 8, 2, 0, 0),
 'ghi': 47.0,
 'hotspots': [{'confidence': 94.0,
               'latitude': -37.4796,
               'longitude': 141.9403,
               'surface_temperature_celcius': 87.0,
               'time': datetime.datetime(2018, 8, 2, 3, 45, 40)},
              {'confidence': 54.0,
               'latitude': -37.491,
               'longitude': 141.936,
               'surface_temperature_celcius': 40.0,
               'time': datetime.datetime(2018, 8, 2, 3, 45)}],
 'max_wind_speed': 5.1,
 'precipitation': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'_id': ObjectId('60aa4d77f1c7e6844f090c71'),
 'air_temperature_celcius': 5.0,
 'date': datetime.datetime(2018, 6, 30, 0, 0),
 'ghi': 48.0,
 'hotspots': [{'confidence': 78.0,
               'latitude': -36.834,
               'longitude': 142.524,
              

j. Find the records with a 24-hour precipiation record between 0.20 and 0.35.

In [41]:
result = collection.aggregate(
    [{'$match': {'precipitation_type': 'G', 'precipitation': {'$gte': 0.20, '$lte': 0.35}}}])

for each in result:
    pprint(each)


{'_id': ObjectId('60aa4e725841cf5659939f10'),
 'air_temperature_celcius': 19.0,
 'date': datetime.datetime(2018, 1, 13, 0, 0),
 'ghi': 157.0,
 'max_wind_speed': 18.1,
 'precipitation': 0.31,
 'precipitation_type': 'G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'_id': ObjectId('60aa4e725841cf5659939f5b'),
 'air_temperature_celcius': 17.0,
 'date': datetime.datetime(2018, 3, 29, 0, 0),
 'ghi': 146.0,
 'hotspots': [{'confidence': 69.0,
               'latitude': -34.2648,
               'longitude': 141.6325,
               'surface_temperature_celcius': 51.0,
               'time': datetime.datetime(2018, 3, 29, 0, 48, 40)}],
 'max_wind_speed': 21.0,
 'precipitation': 0.24,
 'precipitation_type': 'G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'_id': ObjectId('60aa4e725841cf5659939f71'),
 'air_temperature_celcius': 20.0,
 'date': datetime.datetime(2018, 4, 20, 0, 0),
 'ghi': 166.0,
 'hotspots': [{'confidence': 84.0,
       